In [ ]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark
!pip install py4j

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:12 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:15 htt

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-01-26 06:35:31--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.21MB/s    in 0.8s    

2021-01-26 06:35:33 (1.21 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Home_Entertainment_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
home_entertainment_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
home_entertainment_df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|  product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+------------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|     179886| RY01SAV7HZ8QO|B00NTI0CQ2|     667358431|Aketek 1080P LED ...|Home Entertainment|          4|            0|          0|   N|                Y|good enough for m...|not the best pict...| 2015-08-31|
|         US|   37293769|R1XX8SDGJ4MZ4L|B00BUCLVZU|     621695622|TiVo Mini with IR...|Home Entertainment|      

In [ ]:
vine_df = home_entertainment_df.select(["review_id","star_rating", "helpful_votes", "total_votes","vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RY01SAV7HZ8QO|          4|            0|          0|   N|                Y|
|R1XX8SDGJ4MZ4L|          5|            0|          0|   N|                N|
|R149Q3B5L33NN5|          5|            0|          0|   N|                Y|
|R2ZVD69Z6KPJ4O|          1|            0|          2|   N|                Y|
|R1DIKG2G33ZLNP|          4|            0|          0|   N|                Y|
|R3L6FGKAW0EYFI|          1|            1|          1|   N|                N|
| RAO0QZH5VC6VI|          1|            0|          0|   N|                Y|
|R25IK0UAHWNB22|          3|            0|          0|   N|                Y|
|R2A9IHKZMTMAL1|          5|            1|          2|   N|                Y|
| R5XVKTHL6SITI|          5|            0|          0|   N|     

In [ ]:
#Total Vote Counts greater than 20
votes_count_df = vine_df.filter("total_votes >=20")
votes_count_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
|R1LV778IDGRJQ6|          1|            4|         23|   N|                Y|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R2LBEAGVJA7HK1|          3|            5|         48|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R3QLX5NODD1OJ4|          1|            9|         25|   N|                Y|
|R2SOOHE0XUQON1|          1|            2|         24|   N|     

In [ ]:
#Helpful votes greater than 50%
helpful_votes_df = votes_count.filter(votes_count["helpful_votes"]/votes_count["total_votes"]>=0.5)
helpful_votes_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

In [ ]:
#Paid Vines

paid_vine_df = helpful_votes.filter(helpful_votes["vine"]=="Y")
paid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3LLICBKRHLRJC|          4|           17|         22|   Y|                N|
| RUD135S06DEVE|          5|           41|         51|   Y|                N|
|R2WG8UL4SGDPDH|          4|           23|         25|   Y|                N|
|R158AOKOA28WS2|          4|           18|         28|   Y|                N|
| RGWV3QLJQXU9X|          5|          332|        380|   Y|                N|
| RD0E1RZSSYGGN|          5|          261|        292|   Y|                N|
|R3LGCTQH5ZMGO0|          5|           61|         68|   Y|                N|
|R2H24YTRLARCZJ|          4|          277|        297|   Y|                N|
| RYYTC9JTG7WPW|          5|          107|        134|   Y|                N|
| RTB8H4YQYTLY3|          4|           58|         62|   Y|     

In [ ]:
#Unpaid Vines

unpaid_vine_df = helpful_votes.filter(helpful_votes["vine"]=="N")
unpaid_vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R1VCSDR38XTALJ|          1|           33|         40|   N|                N|
|R1KMC6B2JRCZ3D|          3|           42|         47|   N|                N|
| RBZE8H6SHVYWW|          3|           77|        110|   N|                N|
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
|R1K00VBT0SMD4Z|          1|           50|         60|   N|                Y|
|R2I9AO0VEBYJES|          4|           47|         57|   N|                Y|
|R3Q7YJX1I5PDMU|          3|           22|         22|   N|                Y|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R2QJI05ZY7DNSR|          4|           59|         73|   N|     

In [ ]:
#Five Star Reviews
five_star_df = helpful_votes_df.filter(helpful_votes_df["star_rating"]==5)
five_star_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3UN4H3BW9YTQA|          5|          358|        388|   N|                Y|
| R2YKRQ8R3B2Q3|          5|           30|         35|   N|                N|
| RJQQO26UPW1PI|          5|           76|         96|   N|                Y|
|R14QWM0LJO6313|          5|           24|         28|   N|                Y|
|R19H4AKN005KL8|          5|           19|         28|   N|                Y|
|R27MJ6EZADWWPX|          5|           34|         37|   N|                Y|
|R2GFDC3KTYASPJ|          5|           94|        118|   N|                Y|
|R25PC9R5T29WSK|          5|           21|         23|   N|                Y|
|R22UO302E0W4DK|          5|           17|         23|   N|                N|
| RDCFYG43XCPNM|          5|           46|         55|   N|     

In [ ]:
#Five Star reviews count
five_star_df.count()

11005

In [ ]:
#Helpful Reviews count
helpful_votes_df.count()

24301

In [ ]:
five_star_df.count() / helpful_votes_df.count()

0.4528620221390066

In [ ]:
#Percentage of paid five star reviews
five_star_df.filter(five_star_df["verified_purchase"] == "Y").count()/helpful_votes_df.filter(helpful_votes_df["verified_purchase"]== "Y").count()

0.45623721881390594

In [ ]:
#Percentage of unpaid five star reviews
five_star_df.filter(five_star_df["verified_purchase"] == "N").count()/helpful_votes_df.filter(helpful_votes_df["verified_purchase"]== "N").count()

0.45058880242407545